In [ ]:
import numpy as np
import statmapper as stm
from sklearn_tda import MapperComplex
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.cluster import AgglomerativeClustering

Load data.

In [ ]:
X = np.loadtxt("human")

Compute Mapper with `sklearn_tda`

In [ ]:
params = {"filters": X[:,2:3], "filter_bnds": np.array([[np.nan,np.nan]]), "colors": X, 
          "resolutions": np.array([24]), "gains": np.array([0.3]), "inp": "point cloud", 
          "clustering": AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=.1)}

In [ ]:
M = MapperComplex(**params).fit(X)

Print Mapper to `.dot` file that can then be processed with the open source [`graphviz`](https://www.graphviz.org/) library.

In [ ]:
stm.print_to_dot(M)

Display Mapper with `networkx` library.

In [ ]:
G = stm.mapper2networkx(M)
nx.draw(G, pos=nx.kamada_kawai_layout(G), 
        node_color=[M.node_info_[node]["colors"][2] for node in G.nodes()])
plt.show()

Define function and topological feature type with which we will compute persistence on the Mapper. In the cell below, the function is defined as the 3rd coordinate of the point cloud and the feature type is downward branch.

In [ ]:
function = X[:,2]
topo = "downbranch"
path = "external_functions/HomologyLocalization"

Compute downbranches and their representative nodes.

In [ ]:
dgm, bnd = stm.compute_topological_features(M, function, topo, path_to_homology_localization=path)

In [ ]:
print(dgm, bnd)

Filter downbranches with significance less than 90% with 100-bootstrap.

In [ ]:
confidence_threshold  = 0.90
bootstrap_iterations  = 100

In [ ]:
sdgm, sbnd = stm.evaluate_significance(dgm,bnd,X,M,function,params,topo,confidence_threshold,bootstrap_iterations)

In [ ]:
print(sdgm, sbnd)

Sort coordinates with respect to their ability to differentiate the first significant downbranch from the rest of the Mapper nodes.

In [ ]:
features, pv = stm.compute_DE_features(X, M, sbnd[0])

In [ ]:
print(features, pv)